In [166]:
import requests 
import pandas as pd
from random import sample 
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns                                                             
import sys 
import math
from networkx.algorithms.community import greedy_modularity_communities

### Generating Data from Transition Matrix

In [29]:
# Arbitrary chosen params
states = ["a","b","c","d"]
state_to_index = {"a":0,"b":1,"c":2,"d":3}
initial_probabilties = [0.1, 0.3, 0.5, 0.1]
transition_matrix = np.asarray([
                                [0.1, 0.9, 0.0, 0.0],
                                [0.5, 0.2, 0.2, 0.1],
                                [0.3, 0.3, 0.3, 0.1],
                                [0.2, 0.2, 0.2, 0.4]
                              ])

In [27]:
# arbitraily decide length of produced sequence
path_length = 10

# generate start state
current_index = np.random.choice(list(range(len(states))), p=initial_probabilties)
current_state = states[index]

generated_state_set = [current_state]

# generate sequential states
for _ in range(path_length-1):
    next_index = np.random.choice(list(range(len(states))), p=transition_matrix[current_index])
    next_state = states[next_index]
    generated_state_set.append(next_state)
    current_index = next_index
    
print("generated state set:", generated_state_set)

generated state set: ['a', 'a', 'b', 'a', 'b', 'c', 'c', 'c', 'a', 'a']


### Calculate LL of Given Sequence

In [31]:
LL_i = 0

# calculate LL of initial state
prev_state = generated_state_set[0]
prev_index = state_to_index[prev_state]
LL_i += math.log(initial_probabilties[prev_index])

# calculate entire sequence probabilty
for current_state in generated_state_set:
    current_index = state_to_index[current_state]
    LL_i += math.log(transition_matrix[prev_index][current_index])
    prev_index = current_index

print("LL =", LL_i)

LL = -15.335564909263686


### Generate Transition Matrix from Data
#### Generate Data

In [85]:
# arbitraily decide length of produced sequence
path_length = 10
number_data_sets = 10000
generated_sets = []

for _ in range(number_data_sets):
    # generate start state
    current_index = np.random.choice(list(range(len(states))), p=initial_probabilties)
    current_state = states[current_index]

    generated_state_set = [current_state]

    # generate sequential states
    for _ in range(path_length-1):
        next_index = np.random.choice(list(range(len(states))), p=transition_matrix[current_index])
        next_state = states[next_index]
        generated_state_set.append(next_state)
        current_index = next_index
    
    generated_sets.append(generated_state_set)

print(generated_sets[:5])

[['a', 'b', 'a', 'a', 'b', 'a', 'b', 'a', 'b', 'd'], ['b', 'b', 'a', 'b', 'c', 'a', 'b', 'b', 'a', 'b'], ['c', 'c', 'b', 'a', 'b', 'a', 'b', 'c', 'b', 'b'], ['c', 'b', 'b', 'd', 'c', 'c', 'b', 'a', 'b', 'a'], ['c', 'c', 'b', 'c', 'a', 'a', 'b', 'a', 'a', 'b']]


note: I think it makes sense to do lapace smoothing for seq length generation, due to dynamics of inserting a gene

#### Generate Transition Matrix for 1st order MC from Data

In [86]:
observed_states = np.sort(np.unique(np.concatenate(generated_sets)))
state_to_index = {observed_states[i]: i for i in range(len(observed_states))}
estimated_transition_matrix = np.asarray([[0.0]*len(observed_states)]*len(observed_states))
estimated_intial_states = np.asarray([0.0]*len(observed_states))

for generated_set in generated_sets:
    current_index = state_to_index[generated_set[0]]
    estimated_intial_states[current_index] += 1.0
    
    for next_state in generated_set[1:]:
        next_index = state_to_index[next_state]
        estimated_transition_matrix[current_index][next_index] += 1.0
        current_index = next_index
        
estimated_intial_states = estimated_intial_states/sum(estimated_intial_states)    
for i in range(len(estimated_transition_matrix)):
    estimated_transition_matrix[i] = estimated_transition_matrix[i]/(sum(estimated_transition_matrix[i]) if sum(estimated_transition_matrix[i])>0 else 1)
    
print(estimated_intial_states)
print(estimated_transition_matrix)

[0.0992 0.2953 0.505  0.1005]
[[0.09846978 0.90153022 0.         0.        ]
 [0.50030027 0.20128842 0.19883169 0.09957963]
 [0.30027685 0.30188261 0.29784053 0.1       ]
 [0.20565132 0.19561423 0.1976871  0.40104735]]


In [92]:
print("Difference Between Observed and Estimated Initial State Probabilties")
print(initial_probabilties - estimated_intial_states)

print("Difference Between Observed and Estimated Transition Matrix")
print(transition_matrix - estimated_transition_matrix)

Difference Between Observed and Estimated Transition Matrix
[ 0.0008  0.0047 -0.005  -0.0005]
[[ 0.00153022 -0.00153022  0.          0.        ]
 [-0.00030027 -0.00128842  0.00116831  0.00042037]
 [-0.00027685 -0.00188261  0.00215947  0.        ]
 [-0.00565132  0.00438577  0.0023129  -0.00104735]]


#### Calculate LL of Transition Matrix for all Observed Sequences Against the Estimated Markov Chain

In [90]:
LL = 0

for generated_set in generated_sets:
    LL_i = 0
    current_index = state_to_index[generated_set[0]]
    LL_i += math.log(estimated_intial_states[current_index])
    
    for next_state in generated_set[1:]:
        next_index = state_to_index[next_state]
        LL_i += math.log(estimated_transition_matrix[current_index][next_index])
        current_index = next_index
        
    LL += LL_i
print(LL)

-100687.80941603435


### Generate Sequences that have Conserved Core Regions


In [177]:
core_region_1 = ['a1','a2','a3','a4']
core_region_2 = ['b1','b2','b3','b4']
core_region_3 = ['c1','c2','c3','c4']
variable_elements = ['d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

genome_length = 60
number_genomes = 1000
generated_sets = []

for _ in range(number_genomes):
    # 1. randomly pick subset of and mix up variable elements
    genome_template = np.random.choice(variable_elements, genome_length - (len(core_region_1)+len(core_region_2)+len(core_region_3)))

    # 2. insert mixed up core elements
    mixed_core_1 = np.random.choice(core_region_1, len(core_region_1), replace = False)
    mixed_core_2 = np.random.choice(core_region_2, len(core_region_2), replace = False)
    mixed_core_3 = np.random.choice(core_region_3, len(core_region_3), replace = False)
    
    mixed_core_1 = core_region_1
    mixed_core_2 = core_region_2
    mixed_core_3 = core_region_3
    
    # 3. chose where to insert core genomes
    insertion_locations = np.array((range(len(genome_template)-1)))

    insertion_location = np.random.choice(insertion_locations)
    genome_template = np.concatenate([genome_template[:insertion_location],mixed_core_1,genome_template[insertion_location+1:]])
    insertion_locations[insertion_location:] += len(mixed_core_1)
    #print(genome_template, insertion_locations)

    insertion_location = np.random.choice(insertion_locations)
    genome_template = np.concatenate([genome_template[:insertion_location],mixed_core_2,genome_template[insertion_location+1:]])
    insertion_locations[insertion_location:] += len(mixed_core_2)
    #print(genome_template, insertion_locations)

    insertion_location = np.random.choice(insertion_locations)
    genome_template = np.concatenate([genome_template[:insertion_location],mixed_core_3,genome_template[insertion_location+1:]])
    
    generated_sets.append(genome_template)


#### First order markov chain

In [178]:
observed_states = np.sort(np.unique(np.concatenate(generated_sets)))
state_to_index = {observed_states[i]: i for i in range(len(observed_states))}
estimated_transition_matrix = np.asarray([[0.0]*len(observed_states)]*len(observed_states))
estimated_intial_states = np.asarray([0.0]*len(observed_states))

for generated_set in generated_sets:
    current_index = state_to_index[generated_set[0]]
    estimated_intial_states[current_index] += 1.0
    
    for next_state in generated_set[1:]:
        next_index = state_to_index[next_state]
        estimated_transition_matrix[current_index][next_index] += 1.0
        current_index = next_index
        
estimated_intial_states = estimated_intial_states/sum(estimated_intial_states)    
for i in range(len(estimated_transition_matrix)):
    estimated_transition_matrix[i] = estimated_transition_matrix[i]/(sum(estimated_transition_matrix[i]) if sum(estimated_transition_matrix[i])>0 else 1)
    
print(estimated_intial_states)
print(estimated_transition_matrix)

[0.022 0.    0.    0.    0.018 0.    0.    0.    0.013 0.    0.    0.
 0.039 0.033 0.045 0.032 0.035 0.042 0.025 0.055 0.037 0.035 0.046 0.036
 0.043 0.032 0.051 0.048 0.054 0.053 0.036 0.038 0.05  0.033 0.049]
[[0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.02164948 0.         0.         ... 0.04175258 0.04484536 0.03453608]
 [0.02302459 0.         0.         ... 0.04343276 0.03192046 0.04343276]
 [0.01715177 0.         0.         ... 0.04365904 0.03846154 0.03898129]]


In [179]:
LL = 0

for generated_set in generated_sets:
    LL_i = 0
    current_index = state_to_index[generated_set[0]]
    LL_i += math.log(estimated_intial_states[current_index])
    
    for next_state in generated_set[1:]:
        next_index = state_to_index[next_state]
        LL_i += math.log(estimated_transition_matrix[current_index][next_index])
        current_index = next_index
        
    LL += LL_i
print(LL)

-155611.91854683484


In [180]:
G_markov = nx.from_numpy_matrix(estimated_transition_matrix)
groupings = greedy_modularity_communities(G_markov, weight=None)

for g in range(30):
    print("GROUP ",g)
    for i in groupings[g]:
        print(observed_states[i])
    print()
    print()

GROUP  0
a2
a3


GROUP  1
b2
b3


GROUP  2
c2
c3


GROUP  3
a1


GROUP  4
a4


GROUP  5
b1


GROUP  6
b4


GROUP  7
c1


GROUP  8
c4


GROUP  9
d


GROUP  10
e


GROUP  11
f


GROUP  12
g


GROUP  13
h


GROUP  14
i


GROUP  15
j


GROUP  16
k


GROUP  17
l


GROUP  18
m


GROUP  19
n


GROUP  20
o


GROUP  21
p


GROUP  22
q


GROUP  23
r


GROUP  24
s


GROUP  25
t


GROUP  26
u


GROUP  27
v


GROUP  28
w


GROUP  29
x


